# Chapter 5 - Deep learning for computer vision

## Introduction to convnets

In this section, we are going to take a look at convolutional networks and talk about why they have been so sucessful at computer vision tasks. To get started, we are going to see how we can improve the digit recognition from the MNIST dataset from chapter 2.

In [1]:
from tensorflow.keras import layers
from tensorflow.keras import models

## The difference

The section below is where we start to depart from the example from chapter 2. Instead of a network consisting of just densely connected layers, there is a layer of Conv2D layers that I like to think of as a pre-processing step for the densely connected layers. Convolutional layers excell at finding patterns in images because they apply a set of filters over the entire image where dense layers apply their filters indescriminately. I know this is probably a bad example but it's how I like to think about it.

As we can see in the example below, Maxpooling2D effectifely halves the size of the netweek each time it is called. This is be design as MaxPooling is intended to aggresively downsample the feature maps from the preceding Conv2D layer. The MaxPooling operation is similar to the Convolution operation except it does a max tensor operation with a stride of 2 instead of 3, hence the halving. This downsampling decreases the total amount of tensors in a network thereby decreasing the overfitting of the model. The downsampling helps the model generalize features by looking at larger windows.

In [38]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


## Conventional methods

After the convolutional layers we add two densely connected layers. At first, this did not make sense to me. But then I considered what each layer type excells at. Convolutional layers excell at finding image features and converting the image (2D tensor) to a list of probabilities that each feature exists. Dense layers excell at finding correlations in numerical data. So this network is doing feature extraction on the image using convolutional layers and then doing classification on the resulting feature probabilities. Smart.

In [40]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

## Normalizing the data

Again we have an example of data normalization but in this case its an image so normalization is pretty straighforward because we know the bounds of the numbers inside. First we cast all of the values contained in the image arrays to floats (to preserve precision after normalization) and then we divide by 255 to map all values to a value between 0 to 1. Next we see to_categorical being used to set test and train labels. This basically converts the result integer into a binary array that can be understood by the algorithm. 

For example a label array containing:  
`[1, 3]`   
would be converted to:   
`[[0, 1, 0, 0], 
 [0, 0, 0, 1]]`

In [43]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [44]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [45]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [46]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=["accuracy"])

In [47]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 106us/sample - loss: 0.1650 - accuracy: 0.9471
Epoch 2/5
60000/60000 [==============================] - 4s 63us/sample - loss: 0.0469 - accuracy: 0.9854
Epoch 3/5
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0324 - accuracy: 0.9902
Epoch 4/5
60000/60000 [==============================] - 4s 59us/sample - loss: 0.0254 - accuracy: 0.9919
Epoch 5/5
60000/60000 [==============================] - 4s 59us/sample - loss: 0.0195 - accuracy: 0.9937


## Results

After compiling and training the model we see that just by including the convolutional layers we have increased the accuracy of the model to 99% versus the 97% we got in chapter two. A marked improvement from smarter model architecture versus trying to brute force improvements with more data 

In [16]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 62us/sample - loss: 0.0346 - accuracy: 0.9923


In [17]:
test_acc

0.9923

## Training a convnet using a small datasets

In this example, we will be using a convnet and data augmentation to train a convolutional model with a small dataset.

In [8]:
import os, shutil

In [11]:
original_dataset_dir = '../datasets/dogs-vs-cats'
base_dir = '../datasets/dogs-vs-cats-small'

In [12]:
os.mkdir(base_dir)

In [13]:
def copyImages(source, destination, irange):    
    dest_cat_dir = os.path.join(destination, 'cats')
    dest_dog_dir = os.path.join(destination, 'dogs')
    
    cfnames = ['cat.{}.jpg'.format(i) for i in irange]
    for cfname in cfnames:
        src = os.path.join(source, cfname)
        dst = os.path.join(dest_cat_dir, cfname)
        shutil.copyfile(src, dst)
    
    dfnames = ['dog.{}.jpg'.format(i) for i in irange]
    for dfname in dfnames:
        src = os.path.join(source, dfname)
        dst = os.path.join(dest_dog_dir, dfname)
        shutil.copyfile(src, dst)

In [14]:
train_dir = os.path.join(base_dir, 'train')

validation_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'test')

In [15]:
os.mkdir(train_dir)

train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

original_train_dir = os.path.join(original_dataset_dir, 'train')

copyImages(original_train_dir, train_dir, range(1000))

print('total training cat images: ', len(os.listdir(train_cats_dir)))
print('total training dog images: ', len(os.listdir(train_dogs_dir)))

total training cat images:  1000
total training dog images:  1000


In [16]:
os.mkdir(validation_dir)

validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

original_train_dir = os.path.join(original_dataset_dir, 'train')

copyImages(original_train_dir, validation_dir, range(1000, 1500))

print('total validation cat images: ', len(os.listdir(validation_cats_dir)))
print('total validation dog images: ', len(os.listdir(validation_dogs_dir)))

total validation cat images:  500
total validation dog images:  500


In [17]:
os.mkdir(test_dir)

test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

original_train_dir = os.path.join(original_dataset_dir, 'train')

copyImages(original_train_dir, test_dir, range(1500, 2000))

print('total test cat images: ', len(os.listdir(test_cats_dir)))
print('total test dog images: ', len(os.listdir(test_dogs_dir)))

total test cat images:  500
total test dog images:  500


In [18]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

In [20]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

## Data preprocessing

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [23]:
history = model.fit(train_generator,
                   steps_per_epoch=100,
                   epochs=30,
                   validation_data=validation_generator,
                   validation_steps=50)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/30
100/100 [==============================] - 8s 84ms/step - loss: 0.6920 - acc: 0.5380 - val_loss: 0.6782 - val_acc: 0.5150
Epoch 2/30
100/100 [==============================] - 6s 62ms/step - loss: 0.6701 - acc: 0.5960 - val_loss: 0.6590 - val_acc: 0.5930
Epoch 3/30
100/100 [==============================] - 6s 64ms/step - loss: 0.6349 - acc: 0.6455 - val_loss: 0.6151 - val_acc: 0.6710
Epoch 4/30
100/100 [==============================] - 7s 66ms/step - loss: 0.5813 - acc: 0.6940 - val_loss: 0.6232 - val_acc: 0.6430
Epoch 5/30
100/100 [==============================] - 7s 66ms/step - loss: 0.5425 - acc: 0.7240 - val_loss: 0.5971 - val_acc: 0.6720
Epoch 6/30
100/100 [==============================] - 6s 63ms/step - loss: 0.5044 - acc: 0.7500 - val_loss: 0.5590 - val_acc: 0.7200
Epoch 7/30
100/100 [==============================] - 6s 63ms/step - loss: 0.4783 - acc: 0.7745 - val_loss: 0